In [1]:
from bs4 import BeautifulSoup
import mechanicalsoup
import requests
import time, os
from selenium import webdriver
import math
import pickle
import re
import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tummee.com/login?")

cookies = pickle.load(open("cookies.pickle", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://www.tummee.com/sequences/teachers/daily")

In [3]:
f = open("power_urls", "rb")
power_urls = pickle.load(f)
f.close()

In [4]:
def get_poses(url): 
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try: 
        title = re.sub(r"[\t\n0-9.]", "", soup.find("h1").text.strip())
        find_poses = soup.find_all("a", class_="english-title")
        poses = [re.sub(r"[\t\n0-9.]", "", i.text.strip()) for i in find_poses]
        class_type = "Power"
    except AttributeError:
        info = {"Title": np.nan}
    else: 
        info = {"Title": title, "Poses": poses, "Class Type": class_type}
    return info

In [5]:
power_classes = [get_poses(url) for url in power_urls]

In [7]:
df = pd.DataFrame(power_classes)
df

,Title,Poses,Class Type
0,←Beginner Yoga,"[Reclined Mountain Pose, Cat Cow Pose Forearms...",Power
1,←Candlelight Flow /,"[Boat Pose Variation Knees Bent Arms Side, Hal...",Power
2,←Balancing vinyasa,"[Child Pose, Side Plank Pose Variation (One Kn...",Power
3,←HIIT Yoga,"[Wide Child Pose, Child Pose Flow, Repeat , Th...",Power
4,←Vinyasa Flow,"[Cat Cow Pose, Cat Cow Pose Downward Facing Do...",Power
...,...,...,...
1473,←core /,"[Cat Cow Pose, Hovering Cat Pose, Thread The N...",Power
1474,← de Agosto Power,"[Salutation Seal Pose, Easy Pose Neck Side Str...",Power
1475,←dancing yoga,"[Child Pose, Child Pose Variation Both Arms To...",Power
1476,←Core workout,"[Warrior Pose Flow, Pyramid Pose Flow, Intense...",Power


In [8]:
f = open("power_df", "wb")
pickle.dump(df, f)
f.close()